In [14]:
import scipy as sp
from scipy import stats
import numpy as np


In [54]:
help(map)

Help on class map in module builtins:

class map(object)
 |  map(func, *iterables) --> map object
 |  
 |  Make an iterator that computes the function using arguments from
 |  each of the iterables.  Stops when the shortest iterable is exhausted.
 |  
 |  Methods defined here:
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __reduce__(...)
 |      Return state information for pickling.



In [23]:
a = np.arange(125).reshape(5,5,5)

In [39]:
s, i, r, p, st =  np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan),np.full((5,5), np.nan)

In [64]:
stats.linregress(a[ind,ind,:], a[ind,ind,:]*2)


ValueError: m has more than 2 dimensions

In [200]:
def multiple_linregress(x, y):
    x_mean = np.mean(x, axis=2, keepdims=True)
    x_norm = x - x_mean
    y_mean = np.mean(y, axis=2, keepdims=True)
    y_norm = y - y_mean

    slope = (np.einsum('ijk,ijk->ij', x_norm, y_norm) /
             np.einsum('ijk,ijk->ij', x_norm, x_norm))
    del x_norm
    intercept = y_mean[:, :,0] - slope * x_mean[:, :,0]
    del x_mean
    y_pred_norm = (intercept[:,:,np.newaxis] + slope[:,:,np.newaxis] * x) - y_mean
    R2 = (np.einsum('ijk,ijk->ij', y_pred_norm, y_pred_norm) /
          np.einsum('ijk,ijk->ij', y_norm, y_norm))
    
    return R2, slope, intercept

m = 2500
n = 2500
x = np.random.rand(m, n, 9)
y = np.random.rand(m, n, 9)
r2,s,i = multiple_linregress(x, y)

In [206]:
np.max(r2)


0.97248746714544976